In [1]:
import torch
from torch import nn,optim
import torch.nn.functional
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os as os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class IsolationTreeEnsemble:
    def __init__(self, num_trees, max_depth):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.trees = [IsolationTree(max_depth) for _ in range(num_trees)]

    def fit(self, X):
        for tree in self.trees:
            tree.fit(X)

    def predict(self, X):
        depths = np.array([tree.depth(X) for tree in self.trees])
        avg_depths = depths.mean(axis=0)
        scores = 2.0 ** (-avg_depths / self.avg_depth(X))
        return scores

    def avg_depth(self, X):
        depths = np.array([tree.depth(X) for tree in self.trees])
        avg_depths = depths.mean(axis=0)
        return np.maximum(1e-12, avg_depths)

class IsolationTree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, current_depth=0):
        if current_depth >= self.max_depth or len(X) <= 1:
            return Node(X)

        split_attribute = np.random.choice(X.shape[1])
        split_value = np.random.uniform(X[:, split_attribute].min(), X[:, split_attribute].max())

        left_mask = X[:, split_attribute] < split_value
        right_mask = ~left_mask

        left_subtree = self.fit(X[left_mask], current_depth + 1)
        right_subtree = self.fit(X[right_mask], current_depth + 1)

        return Node(X, split_attribute, split_value, left_subtree, right_subtree)

    def depth(self, X, node=None, current_depth=0):
        if node is None:
            node = self.root

        if node.is_leaf:
            return current_depth

        if X[node.split_attribute] < node.split_value:
            return self.depth(X, node.left, current_depth + 1)
        else:
            return self.depth(X, node.right, current_depth + 1)

class Node:
    def __init__(self, X, split_attribute=None, split_value=None, left=None, right=None):
        self.split_attribute = split_attribute
        self.split_value = split_value
        self.left = left
        self.right = right
        self.size = len(X)

    @property
    def is_leaf(self):
        return self.left is None and self.right is None
